In [2]:
import cv2
from win32api import GetSystemMetrics
import numpy as np

In [19]:
import numpy as np
import cv2
import pandas as pd

img = cv2.imread("01_connecter_1/3.bmp", cv2.IMREAD_COLOR)

def dist(P, A, B):
    area = abs ( (A[0] - P[0]) * (B[1] - P[1]) - (A[1] - P[1]) * (B[0] - P[0]) )
    AB = ( (A[0] - B[0]) ** 2 + (A[1] - B[1]) ** 2 ) ** 0.5
    return ( area / AB )

lineList = []
img2 = img.copy()
# 그레이 스케일로 변환 및 엣지 검출 ---①
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150 )
_, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
binary2 = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel,iterations = 3)
contours,_ = cv2.findContours(binary2, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)


# 확율 허프 변환 적용 ---②
lines = cv2.HoughLinesP(edges, 0.5, np.pi/180, 1, None, 1, 1)

for line in lines:
    lineList.append(line[0])

listColumns = ['x1','y1','x2','y2']
lineDF = pd.DataFrame(lineList, columns=listColumns)
p1 = lineDF.sort_values(by=['x1'])
p1 = p1[p1.x1 < p1.iloc[0,0]+10]
p1 = p1.sort_values(by=['y1'])
p2 = lineDF.sort_values(by=['x1'])
p2 = p2[p2.x1 < p2.iloc[0,0]+10]
p2 = p2.sort_values(by=['y1'], ascending=False)
p4 = lineDF.sort_values(by=['x1'], ascending=False)
p4 = p4[p4.x1 > p4.iloc[0,0]-10]
p4 = p4.sort_values(by=['y1'], ascending=False)
p3 = lineDF.sort_values(by=['x1'], ascending=False)
p3 = p3[p3.x1 > p3.iloc[0,0]-10]
p3 = p3.sort_values(by=['y1'])

p1_x = p1.iloc[0,0]
p1_y = p1.iloc[0,1]
p2_x = p2.iloc[0,0]
p2_y = p2.iloc[0,1]
p3_x = p3.iloc[0,0]
p3_y = p3.iloc[0,1]
p4_x = p4.iloc[0,0]
p4_y = p4.iloc[0,1]

cLineX = [p1_x, (p1_y + p2_y)/2]
cLineY = [p3_x, (p3_y + p4_y)/2]
cv2.line(img2, (int(p1_x), int((p1_y + p2_y)/2)), (int(p3_x), int((p3_y + p4_y)/2)), (0,255,0),2)
cv2.line(img2, (p1.iloc[0,0], p1.iloc[0,1]), (p2.iloc[0,0], p2.iloc[0,1]), (0,0,255), 2)
cv2.line(img2, (p2.iloc[0,0], p2.iloc[0,1]), (p4.iloc[0,0], p4.iloc[0,1]), (0,0,255), 2)
cv2.line(img2, (p4.iloc[0,0], p4.iloc[0,1]), (p3.iloc[0,0], p3.iloc[0,1]), (0,0,255), 2)
cv2.line(img2, (p3.iloc[0,0], p3.iloc[0,1]), (p1.iloc[0,0], p1.iloc[0,1]), (0,0,255), 2)

contourList=[]
for cnt in contours:
    area = cv2.contourArea(cnt)
    if area > 200 :
        M = cv2.moments(cnt)
        cX = int(M['m10'] / M['m00'] + 1e-5)
        cY = int(M['m01'] / M['m00'] + 1e-5)
        p = [cX, cY]
        d = dist(p, cLineX, cLineY)
        if d < 130 :
            contourList.append(p)
            cv2.drawContours(img2, [cnt], 0, (0,255, 0), 2)
            cv2.circle(img2, (cX, cY), 2, (255, 0, 0), -1) 
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            cv2.drawContours(img2,[box],0,(255,127,127),2)

contourDF = pd.DataFrame(contourList, columns=['x','y'])
contourDF = contourDF.sort_values(by=['x', 'y'])
contourUP = contourDF[contourDF.y < (cLineX[1]+cLineY[1])/2]
contourDOWN = contourDF[contourDF.y > (cLineX[1]+cLineY[1])/2]

for i in range(len(contourUP)):
    cv2.putText(img2, str(i+1), (contourUP.iloc[i,0],contourUP.iloc[i,1]-30),cv2.FONT_HERSHEY_COMPLEX, 1, (255,127,127), 2)
for i in range(len(contourDOWN)):
    cv2.putText(img2, str(i+1), (contourDOWN.iloc[i,0],contourDOWN.iloc[i,1]-30),cv2.FONT_HERSHEY_COMPLEX, 1, (255,127,127), 2)

(h, w) = img2.shape[:2]

#위도우 크기
x = GetSystemMetrics(0) / w
y = GetSystemMetrics(1) / h

if x<y:
    img2 = cv2.resize(img2, (0,0), fx=x, fy=x)
else:
    img2 = cv2.resize(img2, (0,0), fx=y, fy=y)

cv2.imshow("img", img2)
cv2.imshow("edges", edges)
cv2.imshow("binary", binary)
cv2.imshow("binary2", binary2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
import numpy as np
import cv2
import pandas as pd
for picNum in range(1, 12):
    img = cv2.imread("01_connecter_1/"+str(picNum)+".bmp", cv2.IMREAD_COLOR)

    # 직선과 점 사이의 거리 측정 함수
    def dist(P, A, B):
        area = abs ( (A[0] - P[0]) * (B[1] - P[1]) - (A[1] - P[1]) * (B[0] - P[0]) )
        AB = ( (A[0] - B[0]) ** 2 + (A[1] - B[1]) ** 2 ) ** 0.5
        return ( area / AB )

    
    img2 = img.copy()

    cv2.putText(img2, str(picNum), (50,50),cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)

    # 그레이 스케일로 변환 및 엣지 검출 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150 )
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    contours,_ = cv2.findContours(binary, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)

    # 확율 허프 변환 적용
    lines = cv2.HoughLinesP(edges, 0.5, np.pi/180, 1, None, 1, 1) # 최대한 많이 잡음

    lineList = []
    for line in lines:
        lineList.append(line[0]) # 허프라인들의 좌표 저장

    listColumns = ['x1','y1','x2','y2']
    lineDF = pd.DataFrame(lineList, columns=listColumns)
    # x최소 y최소
    p1 = lineDF.sort_values(by=['x1'])
    p1 = p1[p1.x1 < p1.iloc[0,0]+15]
    p1 = p1.sort_values(by=['y1'])
    # x최소 y최대
    p2 = lineDF.sort_values(by=['x1'])
    p2 = p2[p2.x1 < p2.iloc[0,0]+15]
    p2 = p2.sort_values(by=['y1'], ascending=False)
    # x최대 y최소
    p4 = lineDF.sort_values(by=['x1'], ascending=False)
    p4 = p4[p4.x1 > p4.iloc[0,0]-15]
    p4 = p4.sort_values(by=['y1'], ascending=False)
    # x최대 y최대
    p3 = lineDF.sort_values(by=['x1'], ascending=False)
    p3 = p3[p3.x1 > p3.iloc[0,0]-15]
    p3 = p3.sort_values(by=['y1'])

    p1_x = p1.iloc[0,0]
    p1_y = p1.iloc[0,1]
    p2_x = p2.iloc[0,0]
    p2_y = p2.iloc[0,1]
    p3_x = p3.iloc[0,0]
    p3_y = p3.iloc[0,1]
    p4_x = p4.iloc[0,0]
    p4_y = p4.iloc[0,1]

    # dist함수에 쓰기위한 변수
    cLineX = [p1_x, (p1_y + p2_y)/2]
    cLineY = [p3_x, (p3_y + p4_y)/2]

    # 사각형 그리기
    cv2.line(img2, (p1.iloc[0,0], p1.iloc[0,1]), (p2.iloc[0,0], p2.iloc[0,1]), (0,0,255), 2)
    cv2.line(img2, (p2.iloc[0,0], p2.iloc[0,1]), (p4.iloc[0,0], p4.iloc[0,1]), (0,0,255), 2)
    cv2.line(img2, (p4.iloc[0,0], p4.iloc[0,1]), (p3.iloc[0,0], p3.iloc[0,1]), (0,0,255), 2)
    cv2.line(img2, (p3.iloc[0,0], p3.iloc[0,1]), (p1.iloc[0,0], p1.iloc[0,1]), (0,0,255), 2)
    # 사각형의 가로 중심선, dist함수에서 비교할 선
    cv2.line(img2, (int(p1_x), int((p1_y + p2_y)/2)), (int(p3_x), int((p3_y + p4_y)/2)), (0,255,0),2)

    # 컨투어 그리기
    contourList=[]
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 1350 :
            M = cv2.moments(cnt)
            cX = int(M['m10'] / M['m00'] + 1e-5)
            cY = int(M['m01'] / M['m00'] + 1e-5)
            p = [cX, cY] # dist함수에서 비교할 점
            d = dist(p, cLineX, cLineY)
                        if d < 130 :
                              
            if d < 130 :
                contourList.append(p)
                cv2.drawContours(img2, [cnt], 0, (0,255, 0), 2)
                cv2.circle(img2, (cX, cY), 2, (255, 0, 0), -1) 
                rect = cv2.minAreaRect(cnt)
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                cv2.drawContours(img2,[box],0,(255,127,127),2)

    contourDF = pd.DataFrame(contourList, columns=['x','y'])
    contourDF = contourDF.sort_values(by=['x', 'y'])
    contourUP = contourDF[contourDF.y < (cLineX[1]+cLineY[1])/2]
    contourDOWN = contourDF[contourDF.y > (cLineX[1]+cLineY[1])/2]

    for i in range(len(contourUP)):
        cv2.putText(img2, str(i+1), (contourUP.iloc[i,0],contourUP.iloc[i,1]-30),cv2.FONT_HERSHEY_COMPLEX, 1, (255,127,127), 2)
    for i in range(len(contourDOWN)):
        cv2.putText(img2, str(i+1), (contourDOWN.iloc[i,0],contourDOWN.iloc[i,1]-30),cv2.FONT_HERSHEY_COMPLEX, 1, (255,127,127), 2)

    (h, w) = img2.shape[:2]

    #위도우 크기
    x = GetSystemMetrics(0) / w
    y = GetSystemMetrics(1) / h

    if x<y:
        img2 = cv2.resize(img2, (0,0), fx=x, fy=x)
    else:
        img2 = cv2.resize(img2, (0,0), fx=y, fy=y)

    cv2.imshow("img", img2)
    # cv2.imshow("edges", edges)
    # cv2.imshow("binary", binary)
    # cv2.imshow("binary2", binary2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

IndentationError: unexpected indent (<ipython-input-3-dd7b3115461c>, line 81)

In [ ]:
import cv2
import numpy as np
from win32api import GetSystemMetrics


def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None


def FitToWindowSize(image):
    image_resized = image.copy()
    # 윈도우 크기 얻기
    # from win32api import GetSystemMetrics
    # print("Width =", GetSystemMetrics(0))
    # print("Height =", GetSystemMetrics(1))
    # 이미지 크기 얻기
    print('image {}'.format(image.shape))
    win_w = GetSystemMetrics(0)
    win_h = GetSystemMetrics(1)
    img_h, img_w = image.shape[:2]

    if(img_h > win_h or img_w > win_w):
        rate_width = (win_w / img_w)*0.95
        rate_height = (win_h / img_h)*0.95
        scale = rate_width if (rate_width < rate_height) else rate_height
        image_resized = cv2.resize(image, dsize=(
            0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
    # cv2.imshow('image_resize',image_resized)
    return image_resized

class D1218():
    def __init__(self):
        self.center =[]
        self.Big = []
        self.maxwh = 0
        self.RECT=[]
        self.MMT = []

    def drawconT(self, inputimg, image):
        #중심 점 찾기
        self.gray = cv2.cvtColor(inputimg, cv2.COLOR_BGR2GRAY)
        ret, binary = cv2.threshold(self.gray,100, 255, cv2.THRESH_BINARY_INV)
        edges = cv2.Canny(binary,100,200,apertureSize = 3)
        # 코너 따기
        corners = cv2.goodFeaturesToTrack(edges, 3000, 0.05, 1, blockSize=3, useHarrisDetector=True, k=0.03)
        for i in corners:
            self.RECT.append(list(i[0]))
        self.RECT.sort(key = lambda REC : REC[0])
        # 코너 좌측 우측 나누기
        self.left = self.RECT[:10]
        self.right = self.RECT[-10:]
        # y 값으로 다시 sorting
        self.left.sort(key = lambda REC : REC[1])
        self.right.sort(key = lambda REC : REC[1])
        self.RECT.clear()
        # 4개의 점 추출
        self.RECT.append(self.left[0])
        self.RECT.append(self.left[-1])
        self.RECT.append(self.right[-1])
        self.RECT.append(self.right[0])
        # 사각형 그리기
        cv2.line(image, tuple(self.RECT[0]),tuple(self.RECT[1]),(255,0,0))
        cv2.line(image, tuple(self.RECT[1]),tuple(self.RECT[2]),(255,0,0))   
        cv2.line(image, tuple(self.RECT[2]),tuple(self.RECT[3]),(255,0,0))   
        cv2.line(image, tuple(self.RECT[0]),tuple(self.RECT[3]),(255,0,0)) 
        # 중심점
        self.MMT.append(int(np.mean(self.RECT,axis=0)[0]))
        self.MMT.append(int(np.mean(self.RECT,axis=0)[1]))
        cv2.circle(image,tuple(self.MMT),2,(255,255,0),0)
  
        # contour 
        self.gray = cv2.cvtColor(inputimg, cv2.COLOR_BGR2GRAY)
        ret, binary = cv2.threshold(self.gray, 130, 255, cv2.THRESH_TOZERO)
        contours, hierarchy = cv2.findContours(
            binary, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
        for i in range(len(contours)):
            if cv2.contourArea(contours[i]) < 550 and cv2.contourArea(contours[i]) > 100:
                x, y, w, h = cv2.boundingRect(contours[i])
                M = cv2.moments(contours[i])
                cX = int(M['m10'] / M['m00'])
                cY = int(M['m01'] / M['m00'])
                if w > h+5 and np.abs(cY-self.MMT[1])<75:
                    if cY< image.shape[0]/2:
                        cv2.fillConvexPoly(image,contours[i],(0,0,255))
                    else :
                        cv2.fillConvexPoly(image,contours[i],(255,0,0))
                    #rect그리기
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 1)
                    #중심 그리기
                    cv2.circle(image,(cX,cY),1,(0,255,0),1)
                    self.center.append([cY,cX])
        
        self.center.sort(reverse=True)
        self.downside = self.center[:19]
        self.upside = self.center[19:]
        self.downside.sort(key= lambda side : side[1])
        self.upside.sort(key= lambda side : side[1])
        for i in range(len(self.downside)):
            cv2.putText(image,str(i),(self.downside[i][1]-10,self.downside[i][0]-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,0))
            cv2.putText(image,str(i),(self.upside[i][1]-10,self.upside[i][0]-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,0))

        cv2.line(image, (self.upside[0][1]-20,self.upside[0][0]),(self.upside[-1][1]+20,self.upside[-1][0]),(255,255,0))
        cv2.line(image, (self.downside[0][1]-20,self.downside[0][0]),(self.downside[-1][1]+20,self.downside[-1][0]),(255,255,0))
        
        return image


if __name__ == '__main__':
    
    # 하나만 비교

    # D2 = D1218()
    # src = imread("C:/Users/w/Desktop/OPENCV/01_connecter_1/1.bmp")
    # src = FitToWindowSize(src)
    # dst = src.copy()
    # output = D2.drawconT(src, dst)
    # cv2.imshow('dst', output)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    
    # 여러개 한꺼번에 비교
    for i in range(10):
        src = imread("C:/Users/w/Desktop/OPENCV/01_connecter_1/"+str(i+1)+".bmp")
        D2 = D1218()
        src = FitToWindowSize(src)
        dst = src.copy()
        output = D2.drawconT(src, dst)
        cv2.imshow('dst', output)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        

In [ ]:

for picNum in range(1, 12): # ~11

# 이미지 로드 및 프로세싱
    image = imread('C:/Users/w/Desktop/contents/machineVision/PCB2/01_connecter_1/' + str(picNum) + '.bmp', flags = cv2.IMREAD_COLOR, dtype = np.uint8)
    image = fitSizeImage(image)
    imageCopy = image.copy() # (1080, 1440, 3), dtype = uint8
    cv2.putText(imageCopy, str(picNum), (50, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (255, 0, 0), 2)
    height, width, channel = imageCopy.shape # 1080 1440 3
    imageGray = cv2.cvtColor(imageCopy, cv2.COLOR_BGR2GRAY)
    # 115, 255, cv2.THRESH_BINARY_INV: 모든 이미지에 가장 큰 네모가 잡히는 최저 임계지점
    _, imageBinary = cv2.threshold(imageGray, 115, 255, cv2.THRESH_BINARY_INV)
    # 60, 255, cv2.THRESH_BINARY_INV: 모든 사각형을 잡기 위해 적용되는 임계지점
    _, imageBinary2 = cv2.threshold(imageGray, 60, 255, cv2.THRESH_BINARY_INV)

# 변수 선언
    contourList = [] # 조건을 만족하는 컨투어를 담을 리스트
    momentList = [] # 모멘트 좌표(cX, cY)를 담을 리스트

## 큰 사각형
    contours, _ = cv2.findContours(imageBinary2, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    maxContour = max(contours, key = cv2.contourArea) # maxContour는 3차원 텐서이므로
    dfContour = np.array(maxContour).sum(axis = 1) # sum 함수를 통해 2차행렬로 변환합니다.
    df = pd.DataFrame(dfContour, columns = ['x', 'y'])
    df = df.sort_values(by = ['x', 'y'])
    dfCopy = df.copy() # x, y로 솔팅된 데이터프레임 원본 유지
    dfXMin = dfCopy['x'].min();                   dfXMax = dfCopy['x'].max()
    # boolean indexing
    dfCopy = dfCopy[(dfCopy['x'] < dfXMin + 30) | (dfCopy['x'] > dfXMax - 30)]
    # X, Y 최대 최소값 구하기
    dfYMin = dfCopy['y'].min();                   dfYMax = dfCopy['y'].max()
    cv2.rectangle(imageCopy, (dfXMin, dfYMin), (dfXMax, dfYMax), (0, 255, 0), 1, cv2.LINE_4)
    # 네모에 대한 중간좌표 구하기
    centRectX = np.abs(dfXMax - dfXMin) / 2;      centRectY = np.abs(dfYMax - dfYMin) / 2
    centerX = dfXMin + int(centRectX);            centerY = dfYMin + int(centRectY)
    # 드로잉
    cv2.circle(imageCopy, (centerX, centerY), 1, (255, 0, 255), -1)

### 모멘트
    contours, _ = cv2.findContours(imageBinary, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    if contours is None:
        print("error")
        break
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 500 and area < 630:
            # 모멘트 구하기
            M = cv2.moments(contour)
            cX = int(M['m10'] / M['m00'] + 1e-5)
            cY = int(M['m01'] / M['m00'] + 1e-5)
            if cY > centerY - 75 and cY < centerY + 75:
                momentList.append([cX, cY])
                # 조건을 만족하는 컨투어 및 영역만 담겨져있는 리스트
                contourList.append(contour)
                if cY > centerY:
                    cv2.circle(imageCopy, (cX, cY), 2, (100, 0, 45), -1)
                else:
                    cv2.circle(imageCopy, (cX, cY), 2, (100, 0, 200), -1)            

#### 정렬
    momentDf = pd.DataFrame(momentList, index = np.arange(1, len(momentList) + 1), columns = ['momentX', 'momentY'])
    momentDf.sort_values(by = ['momentY', 'momentX'], inplace = True)
    momentDf.index = np.arange(1, len(momentList) + 1)
    upMomentDf = momentDf[:19] # 위쪽 네모 19개의 데이터프레임
    downMomentDf = momentDf[19:] # 아래쪽 네모 19개의 데이터프레임
    upMomentDf.sort_values(by = 'momentX', inplace = True)
    downMomentDf.sort_values(by = 'momentX', inplace = True)

##### 위 네모 및 아래 네모 19개를 각각 담은 데이터프레임을 가지고 순서를 부여합니다.
    for upItem in range(len(upMomentDf)):
        cv2.putText(imageCopy, str(upItem + 1), (upMomentDf.iloc[upItem, 0] - 10, upMomentDf.iloc[upItem, 1] - 15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (40, 150, 255), 1)
    for downItem in range(len(downMomentDf)):
        cv2.putText(imageCopy, str(downItem + 1), (downMomentDf.iloc[downItem, 0] - 10, downMomentDf.iloc[downItem, 1] - 15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (40, 150, 255), 1)
    for contour in contourList:
        # 사각형 그리기 위한 좌표 구하기
        minX, minY = np.min(contour, axis = (0, 1))
        maxX, maxY = np.max(contour, axis = (0, 1))
        cv2.rectangle(imageCopy, (minX, minY), (maxX, maxY), (50, 255, 255), 1, cv2.LINE_8)

    cv2.imshow('imageCopy', imageCopy)
    if cv2.waitKey(0) & 0xFF == 27:
        cv2.destroyAllWindows()
        break
    else:
        cv2.destroyAllWindows()

In [40]:
import cv2
import numpy as np
 
img = np.zeros(shape=(512,512,3), dtype=np.uint8) + 255
 
ptCenter = img.shape[0]//2, img.shape[1]//2
size = 200, 200
 
# 타원 그리기
cv2.ellipse(img, ptCenter, size, 0, 0, 360, (255, 0, 0))
 
# 중심점이 ptCenter, 크기가 size(x, y)이고, 각도 간격이 45도 마다 좌표 계산
pts1 = cv2.ellipse2Poly(ptCenter, size,  0, 0, 360, delta=45)   # 원을 45도 각도마다 점을 만들기
#print(pts1) # 점의 좌표를 출력
 
# 타원 그리기
cv2.ellipse(img, ptCenter, size, 45, 0, 360, (255, 0, 0))
pts2 = cv2.ellipse2Poly(ptCenter, size, 45, 0, 360, delta=1)	# 원을 45도 각도마다 점을 만들기
 
# 다각형 그리기
cv2.polylines(img, [pts1, pts2], isClosed=True, color=(0, 0, 255))
 
cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()